In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display

### Link to a great video by Finance Explained explaining how the model works:

https://www.youtube.com/watch?v=EEM2YBzH-2U&t

PS: thanks Chatgpt and MistralAI who saved me from wrestling with LaTeX syntax, the rest (the python part) I did myself

### Black-Scholes Formula

The Black-Scholes formula for calculating the price of a European call or put option is given by.

#### Call Option Price:
$$
C = S \cdot N(d_1) - K \cdot e^{-r \cdot T} \cdot N(d_2)
$$

#### Put Option Price:
$$
P = K \cdot e^{-r \cdot T} \cdot N(-d_2) - S \cdot N(-d_1)
$$

Where:
- \( S \) is the current stock price.
- \( K \) is the strike price.
- \( T \) is the time to maturity in years.
- \( r \) is the risk-free interest rate.
- \( σ \) is the volatility of the stock. (Remember: σ is the square root of the variance)
- \( N( ) \) is the cumulative distribution function of the standard normal distribution.

The terms \( d_1 \) and \( d_2 \) are defined as:
$$
d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)(T - t)}{\sigma \sqrt{T - t}}
$$
$$
d_2 = d_1 - \sigma \sqrt{T - t}
$$

The N function is defined as:
$$
N(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-\frac{t^2}{2}} \, dt
$$



In [42]:
def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the price for European call or put options using the Black-Scholes framework.

    Parameters:
        S: Current stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate (annualized)
        sigma: Volatility of the underlying asset (annualized)
        option_type: "call" or "put"

    Returns: Option price
    """
    d1 = ((np.log(S / K) + (r + 0.5 * sigma**2) * T)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

def greeks(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the Greeks.

    Parameters:
        S: Current stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate (annualized)
        sigma: Volatility of the underlying asset (annualized)
        option_type: "call" or "put"

    Returns floats in a dictionnary:
        {Delta, Gamma, Vega, Theta, and Rho}
    """
    d1 = ((np.log(S / K) + (r + 0.5 * sigma**2) * T)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)

    if option_type == "call":
        delta = norm.cdf(d1)  
    else:
        delta = -norm.cdf(-d1)
            
    if option_type == "call":
        theta = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    else: 
        theta = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(-d2)

    if option_type == "call":
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:
        rho = K * T * np.exp(-r * T) * -norm.cdf(-d2)
        

    return {"Delta": delta, "Gamma": gamma, "Vega": vega, "Theta": theta, "Rho": rho}

def plot_payoff(S_range, K, option_type="call"):
    """
    Plot the payoff graph for the option.

    Parameters:
        S_range: Range of stock prices
        K: Strike price
        option_type: "call" or "put"

    Returns:
        A graph
    """
    payoffs = []
    for S in S_range:
        if option_type == "call":
            payoffs.append(max(S - K, 0))
        elif option_type == "put":
            payoffs.append(max(K - S, 0))

    plt.figure(figsize=(8, 4))
    plt.plot(S_range, payoffs, label=f"{option_type.capitalize()} Option Payoff", color="blue")
    plt.axhline(0, color="black", linestyle="--", linewidth=1)
    plt.axvline(K, color="red", linestyle="--", label=f"Strike Price K")
    plt.title(f"Payoff Diagram for a {option_type.capitalize()} Option")
    plt.xlabel("Stock Price at Maturity (S)")
    plt.ylabel("Payoff")
    plt.legend()
    plt.grid()
    plt.show()

def plot_greeks(S, K, T, r, sigma, option_type="call"):
    """
    Plot the Greeks value depending on the underlying's stock price.

    Parameters:
        S: Current stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate (annualized)
        sigma: Volatility of the underlying asset (annualized)
        option_type: "call" or "put"

    Returns:
        5 subplots 
    """
    S_range = np.linspace(0.5 * S, 1.5 * S, 500)
    delta_values, gamma_values, vega_values, theta_values, rho_values = [], [], [], [], []

    for s in S_range:
        greeks_values = greeks(s, K, T, r, sigma, option_type)
        delta_values.append(greeks_values["Delta"])
        gamma_values.append(greeks_values["Gamma"])
        vega_values.append(greeks_values["Vega"])
        theta_values.append(greeks_values["Theta"])
        rho_values.append(greeks_values["Rho"])

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 3, 1)
    plt.plot(S_range, delta_values, label="Delta", color="blue")
    plt.title("Delta")
    plt.xlabel("Stock Price (S)")
    plt.grid(True)

    plt.subplot(2, 3, 2)
    plt.plot(S_range, gamma_values, label="Gamma", color="orange")
    plt.title("Gamma")
    plt.xlabel("Stock Price (S)")
    plt.grid(True)

    plt.subplot(2, 3, 3)
    plt.plot(S_range, vega_values, label="Vega", color="green")
    plt.title("Vega")
    plt.xlabel("Stock Price (S)")
    plt.grid(True)
    
    plt.subplot(2, 3, 4)
    plt.plot(S_range, theta_values, label="Theta", color="red")
    plt.title("Theta")
    plt.xlabel("Stock Price (S)")
    plt.grid(True)
    
    plt.subplot(2, 3, 5)
    plt.plot(S_range, rho_values, label="Rho", color="purple")
    plt.title("Rho")
    plt.xlabel("Stock Price (S)")
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

def interactive_black_scholes(S, K, T, r, sigma, option_type):
    """
    Interactive function to display Black-Scholes option price and Greeks.

    Parameters:
        S: Current stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate (annualized)
        sigma: Volatility of the underlying asset (annualized)
        option_type: "call" or "put"

    Returns:
        5 subplots 
    """
    option_price = black_scholes(S, K, T, r, sigma, option_type)
    S_range = np.linspace(0.5 * S, 1.5 * S, 500)
    g = greeks(S, K, T, r, sigma, option_type)
    print(f"The {option_type} option price is: {round(option_price, 4)}")
    print("\nGreeks:")
    for greek, value in g.items():
        print(f"{greek}: {round(value, 4)}")
    plot_payoff(S_range, K, option_type)
    plot_greeks(S, K, T, r, sigma, option_type)

# Vizualize the interractive charts and play with the widgets 
- User can change the range and step of each variables if needed 

In [45]:
S_slider = widgets.FloatSlider(min=0, max=200, step=0.01, value=100, description='Stock Price (S):')
K_slider = widgets.FloatSlider(min=0, max=200, step=0.01, value=100, description='Strike Price (K):')
T_slider = widgets.FloatSlider(min=0.01, max=10, step=0.01, value=1, description='Time to Maturity (T):')
r_slider = widgets.FloatSlider(min=0.000, max=0.200, step=0.001, value=0.050, description='Risk-free Rate (r):')
sigma_slider = widgets.FloatSlider(min=0.001, max=1.0, step=0.001, value=0.2, description='Volatility (σ):')
option_type_dropdown = widgets.RadioButtons(options=["call", "put"], value="call", description="Option Type")

widgets.interact(interactive_black_scholes, S=S_slider, K=K_slider, T=T_slider, r=r_slider, sigma=sigma_slider, option_type=option_type_dropdown);

interactive(children=(FloatSlider(value=100.0, description='Stock Price (S):', max=200.0, step=0.01), FloatSli…